In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [5]:
### Let's look at informations about tables in database

In [4]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [6]:
###Some queries on tables 

In [18]:
%%sql
SELECT *
FROM album
LIMIT 3;

Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2


In [19]:
%%sql
SELECT *
FROM track
LIMIT 3;

Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99


In [1]:
### Let's write a query to find out which genres sell 
### the most tracks in the USA

In [16]:
%%sql

WITH number_tracks_sold_usa AS 
(
    SELECT c.customer_id,
        c.country,
        t.name track_name,
        t.genre_id
     FROM customer c
     INNER JOIN invoice i ON i.customer_id=c.customer_id
     INNER JOIN invoice_line i_line ON i_line.invoice_id=i.invoice_id
     INNER JOIN track t ON t.track_id=i_line.track_id
     WHERE c.country="USA"
)

SELECT n.country,
    COUNT(n.track_name) tracks_sold,
    CAST(COUNT(n.track_name) AS FLOAT) / (
        SELECT COUNT(*) FROM number_tracks_sold_usa
    ) tracks_percentages,
    g.name genre
FROM number_tracks_sold_usa n
INNER JOIN genre g ON g.genre_id=n.genre_id
GROUP BY 4
ORDER BY 2 DESC
LIMIT 10;
    

Done.


country,tracks_sold,tracks_percentages,genre
USA,561,0.5337773549000951,Rock
USA,130,0.12369172216936251,Alternative & Punk
USA,124,0.11798287345385347,Metal
USA,53,0.05042816365366318,R&B/Soul
USA,36,0.03425309229305423,Blues
USA,35,0.03330161750713606,Alternative
USA,22,0.02093244529019981,Latin
USA,22,0.02093244529019981,Pop
USA,20,0.019029495718363463,Hip Hop/Rap
USA,14,0.013320647002854425,Jazz


Based on the results we should buy albums of artists in the genre Rock, alternative & Punk and Metal. The three artists are respectively: <ul>-Red Tone, Slim Jim Bites, Meteor and the Girls</ul>

Rock represents 53% of the sales which means that we will have to buy more from this genre.

In [17]:
### Analysis of support agents in order to see 
### who is better than others

In [27]:
%%sql
SELECT e.first_name||" "||e.last_name name,
    e.title,
    e.country,
    SUM(i.total) total_dollar
FROM employee e
INNER JOIN customer c ON c.support_rep_id=e.employee_id
INNER JOIN invoice i ON i.customer_id=c.customer_id
GROUP BY 1,2
ORDER BY 4 DESC
LIMIT 10;

Done.


name,title,country,total_dollar
Jane Peacock,Sales Support Agent,Canada,1731.5100000000039
Margaret Park,Sales Support Agent,Canada,1584.0000000000034
Steve Johnson,Sales Support Agent,Canada,1393.920000000002


There are 3 employees and the one with the highest sales score is Jane Peacock.

In [28]:
### Analyzing sales by country

In [29]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


In [31]:
### Albums vs Individual Tracks

In [30]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                     EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.